In [ ]:
# Sommaire


# Packages

In [1]:
from pathlib import Path

import pandas as pd
import pendulum
import scrapy
from scrapy.crawler import CrawlerProcess, CrawlerRunner
from scrapy.utils.log import configure_logging
from scrapy.utils.project import get_project_settings
from twisted.internet import reactor


In [3]:
scrapy.__version__

'2.9.0'

# Settings

In [9]:
DATE_RUN = pendulum.now()
OUTPUT_DATA_URI = Path(Path.cwd(), "data")

FEED_URI = Path(OUTPUT_DATA_URI, f"{DATE_RUN.to_date_string()}-books_data.csv")
print(f"Execution date: {DATE_RUN} \nData URI: {OUTPUT_DATA_URI} \nData output path: {FEED_URI}")

Execution date: 2023-06-16T11:59:33.139309+02:00 
Data URI: /Users/mouslydiaw/Downloads/data_engineer/data 
Data output path: /Users/mouslydiaw/Downloads/data_engineer/data/2023-06-16-books_data.csv


In [12]:
# create the OUTPUT_DATA_URI folder if doesn't exist
OUTPUT_DATA_URI.mkdir(parents=True, exist_ok=True)

In [ ]:
# get_project_settings de Scrapy permet d'obtenir les paramètres de configuration du projet actuel.
# Ces paramètres sont généralement définis dans le fichier settings.py du projet Scrapy.
get_project_settings()

# cf: https://docs.scrapy.org/en/latest/topics/practices.html for more details

In [14]:
DATE_RUN.to_datetime_string()

'2023-06-16 11:59:33'

# Définition de la classe

In [15]:
class BooksSpider(scrapy.Spider):
    start_date = pendulum.now().to_datetime_string()
    print(f"Starting the data scrappng at: {start_date}")
    name = 'books'
    start_urls = [
        'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
        'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
        'https://books.toscrape.com/catalogue/category/books/psychology_26/index.html',
    ]

    def parse(self, response):
        category = response.css('div.page-header > h1::text').get()
        books = response.css('article.product_pod')
        for book in books:
            yield {
                'title': book.css('h3 > a::attr(title)').get(),  # nom des livre
                'price': book.css('div.product_price > p.price_color::text').get(),  # prix
                'category': category,  # catégorie de livre
                "calcul_date": start_date  # execution date
            }
        # passer à la page suivante
        next_page_url = response.css('li.next > a::attr(href)').get()
        if next_page_url:
            yield response.follow(next_page_url, callback=self.parse)

Starting the data scrappng at: 2023-06-16 12:02:30


# Extraction et sauvegarde des résultat

⏩ Dans cette partie du code, j'ai créé une instance de la classe CrawlerProcess de Scrapy. Cette classe est responsable de l'exécution des spiders et de la gestion de l'ensemble du processus d'extraction de données.

Le paramètre settings passé à la classe CrawlerProcess spécifie les paramètres de configuration pour le processus de crawling. Deux paramètres sont utilisés ici :

- "format": Cela spécifie le format de sortie des données extraites. Dans notre cas, il est défini sur 'csv' pour que les données soient exportées au format CSV
- fields: fields to export
- overwrite: flag to know if we replace ou append new data
- FEED_URI: Cela spécifie l'emplacement où les données extraites seront stockées. Dans cet exemple, nous l'avons défini sur 'books_data.csv', ce qui signifie que les données seront enregistrées dans un fichier CSV appelé 'books_data.csv'.


=> Pour plus de paramètres, cf: https://docs.scrapy.org/en/latest/topics/feed-exports.html#std-setting-FEEDS


In [16]:
process = CrawlerProcess(
    settings={
        "FEEDS": {
            FEED_URI: {
                # format data export
                "format": "csv",
                # define the fields to export, their order and their output names
                "fields": ["title", "price", "category"],
                # overwrite the file if it already exists (True) or append to its content (False).
                "overwrite": False,
            }
        }
})

process.crawl(BooksSpider)
process.start(stop_after_crawl=True)

2023-06-16 12:02:33 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-06-16 12:02:33 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.7 (default, Sep 16 2021, 08:50:36) - [Clang 10.0.0 ], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform macOS-10.16-x86_64-i386-64bit
2023-06-16 12:02:33 [scrapy.crawler] INFO: Overridden settings:
{}
2023-06-16 12:02:33 [py.warnings] WARNING: /Users/mouslydiaw/opt/anaconda3/envs/python39/lib/python3.9/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the doc

## Vérification des résultats

In [ ]:
OUTPUT_DATA_URI

In [13]:
data_book = pd.read_csv(FEED_URI, sep=",")
data_book.shape

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mouslydiaw/Downloads/data_engineer/data/2023-06-16-books_data.csv'